In [33]:
!pip install transformers
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/datasets/
!ls
df= pd.read_csv("cleaned_data.csv")
df.drop(index=df.index[0], axis=0, inplace=True)
df.info()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/datasets
 beautfulsoup.py    DataScience			  outputs
 cache_dir	   'fasttext_trained model.bin'   result
 cc.si.300.bin	    file_name.csv		  runs
 cc.si.300.vec	    file.txt			  stem_dictonary.txt
 cleaned_data.csv   news.test			  stopwords.txt
 data.csv	    news.train			  sufix_list.txt
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32294 entries, 1 to 32294
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   News    32294 non-null  object
 1   Label   32294 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 756.9+ KB


In [34]:
df['Label'] = df['Label'].replace(1,0)
df['Label'] = df['Label'].replace(2,1)
df['Label'] = df['Label'].replace(3,2)


In [35]:
tokenizer = BertTokenizer.from_pretrained('keshan/SinhalaBERTo')

In [36]:
token = tokenizer.encode_plus(
    text = df["News"].iloc[0],
        max_length=256,
        truncation=True,
        padding='max_length',
        add_special_tokens=True
)

In [37]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [38]:

def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['News'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [39]:

X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [40]:

labels = np.zeros((len(df), 3))
labels.shape

(32294, 3)

In [41]:
labels[np.arange(len(df)), df['Label'].values] = 0 # one-hot encoded target tensor



In [42]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [43]:

def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels
     

In [44]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset 
     

In [45]:

dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [46]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [47]:
p = 0.8
train_size = int((len(df)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [48]:

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [49]:

from transformers import TFBertModel

In [50]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [51]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [52]:

optim = tf.keras.optimizers.Adam(learning_rate=0.1, decay=0.1)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [53]:

sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [55]:

hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1,
    verbose=1
)

   2/1614 [..............................] - ETA: 19:13:54 - loss: 0.0000e+00 - accuracy: 0.0000e+00

KeyboardInterrupt: ignored

In [ ]:
sentiment_model = tf.keras.models.load_model('sentiment_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Sports', 'International', 'Local']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:
input_text = input('Enter movie review here: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")